In [ ]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix,precision_score, recall_score, f1_score

Read YOLO data & process

In [2]:
# DATA PATH
path = 'final_dataset_withnon'
train_img_path = os.path.join(path, 'images', 'train')
train_lbl_path = os.path.join(path, 'labels', 'train')

valid_img_path = os.path.join(path, 'images', 'val')
valid_lbl_path = os.path.join(path, 'labels', 'val')

test_img_path = os.path.join(path, 'images', 'test')
test_lbl_path = os.path.join(path, 'labels', 'test')

IMG_SIZE = 128
NUM_CLASSES = 4

# path to save processed images
processed_path = os.path.join(path, 'processed')
os.makedirs(processed_path, exist_ok=True)

In [ ]:
# process a dataset by extracting craters from images based on YOLO-format labels and saving them in class-specific directories. 
def process_dataset(img_dir, lbl_dir, output_dir):
    for img_file in os.listdir(img_dir):
        if not img_file.endswith('.jpg'):
            continue # skip non-image files
            
        # Get the corresponding label file
        base_name = os.path.splitext(img_file)[0]
        lbl_file = os.path.join(lbl_dir, f"{base_name}.txt")
        
        # Process single image
        img = Image.open(os.path.join(img_dir, img_file))
        img_w, img_h = img.size
        
        with open(lbl_file, 'r') as f:
            for idx, line in enumerate(f.readlines()):
                class_id, xc, yc, w, h = map(float, line.strip().split())
                # Ensure image bounds
                x1 = int((xc - w/2) * img_w)
                y1 = int((yc - h/2) * img_h)
                x2 = int((xc + w/2) * img_w)
                y2 = int((yc + h/2) * img_h)
                                
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(img_w, x2), min(img_h, y2)
                
                # Crop the crater and resize it
                crater = img.crop((x1, y1, x2, y2))
                crater = crater.resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
                # save
                class_dir = os.path.join(output_dir, str(int(class_id)))
                os.makedirs(class_dir, exist_ok=True)
                crater.save(os.path.join(class_dir, f"{base_name}_{idx}.jpg"))

# process all dataset
process_dataset(train_img_path, train_lbl_path, os.path.join(processed_path, 'train'))
process_dataset(valid_img_path, valid_lbl_path, os.path.join(processed_path, 'val'))
process_dataset(test_img_path, test_lbl_path, os.path.join(processed_path, 'test'))

In [4]:
# define a PyTorch dataset for loading crater images and their corresponding labels.
class CraterDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = []
        self.transform = transform
        
        for class_id in range(NUM_CLASSES):
            class_dir = os.path.join(data_dir, str(class_id))
            for img_file in os.listdir(class_dir):
                self.data.append((os.path.join(class_dir, img_file), class_id))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB') 
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

In [ ]:
# Define data enhancement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Loading data from category folder (using grayscale mode!)
train_generator = train_datagen.flow_from_directory(
    os.path.join(processed_path, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale' # (using grayscale mode!)
)
val_generator = train_datagen.flow_from_directory(
    os.path.join(processed_path, 'val'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'# (using grayscale mode!)
)

Found 1880 images belonging to 4 classes.
Found 413 images belonging to 4 classes.


In [9]:
# test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    os.path.join(processed_path, 'test'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical', 
    shuffle=False,
    color_mode='grayscale'
)

Found 872 images belonging to 4 classes.


Model Test & Prediction

CNN Model

In [ ]:
def create_model():
    model = Sequential()
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        MaxPooling2D((2,2)),

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Parameter settings
NUM_RUNS = 30
class_names = list(test_generator.class_indices.keys())

# Initialize storage structure for metrics
metrics = {
    'classes': {cls: {'precision': [], 'recall': [], 'f1': []} for cls in class_names},
    'macro_avg': {'precision': [], 'recall': [], 'f1': []},
    'weighted_avg': {'precision': [], 'recall': [], 'f1': []},
    'accuracy': []
}

for run in range(NUM_RUNS):
    print(f"\n=========== Training Run {run+1}/{NUM_RUNS} ===========")
    
    # Reset model and random seeds before each training run
    tf.keras.backend.clear_session()
    np.random.seed(run)
    tf.random.set_seed(run)
    
    # Create a new model
    model = create_model()
    
    # Define callbacks
    checkpoint = ModelCheckpoint('best_model.keras', 
                                monitor='val_loss',
                                save_best_only=True,
                                mode='min',
                                verbose=0)
    early_stop = EarlyStopping(monitor='val_loss',
                              patience=5,
                              restore_best_weights=True)
    
    # Train the model
    model.fit(
        train_generator,
        epochs=30,
        validation_data=val_generator,
        callbacks=[checkpoint, early_stop],
        verbose=1  # Adjust verbosity as needed
    )
    
    # Load the best model
    best_model = load_model('best_model.keras')
    
    # Predict on the test set
    y_pred = best_model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes
    
    # Generate classification report
    report = classification_report(y_true, y_pred_classes, 
                                 target_names=class_names,
                                 output_dict=True)
    
    # Collect metrics for each class
    for cls in class_names:
        metrics['classes'][cls]['precision'].append(report[cls]['precision'])
        metrics['classes'][cls]['recall'].append(report[cls]['recall'])
        metrics['classes'][cls]['f1'].append(report[cls]['f1-score'])
    
    # Collect macro and weighted averages
    metrics['macro_avg']['precision'].append(report['macro avg']['precision'])
    metrics['macro_avg']['recall'].append(report['macro avg']['recall'])
    metrics['macro_avg']['f1'].append(report['macro avg']['f1-score'])
    
    metrics['weighted_avg']['precision'].append(report['weighted avg']['precision'])
    metrics['weighted_avg']['recall'].append(report['weighted avg']['recall'])
    metrics['weighted_avg']['f1'].append(report['weighted avg']['f1-score'])
    
    # Collect accuracy
    metrics['accuracy'].append(report['accuracy'])



=========== Training Run 1/30 ===========


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 16s 237ms/step - accuracy: 0.8143 - loss: 0.8051 - val_accuracy: 0.8959 - val_loss: 0.3345
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - accuracy: 0.8501 - loss: 0.4581 - val_accuracy: 0.9104 - val_loss: 0.3177
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 14s 239ms/step - accuracy: 0.8546 - loss: 0.4079 - val_accuracy: 0.9056 - val_loss: 0.2490
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 253ms/step - accuracy: 0.8636 - loss: 0.3809 - val_accuracy: 0.9128 - val_loss: 0.2690
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 251ms/step - accuracy: 0.8598 - loss: 0.3931 - val_accuracy: 0.9177 - val_loss: 0.2273
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 252ms/step - accuracy: 0.8843 - loss: 0.3568 - val_accuracy: 0.9298 - val_loss: 0.2116
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 252ms/step - accuracy: 0.8619 - loss: 0.3535 - val_accuracy: 0.9298 - val_loss: 0.2054
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 251ms/step - accuracy: 0.8932 - loss: 0.3156 - val_accu

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 16s 249ms/step - accuracy: 0.7973 - loss: 0.8437 - val_accuracy: 0.9007 - val_loss: 0.2542
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 253ms/step - accuracy: 0.8580 - loss: 0.4089 - val_accuracy: 0.9128 - val_loss: 0.2140
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 14s 245ms/step - accuracy: 0.8679 - loss: 0.3899 - val_accuracy: 0.9153 - val_loss: 0.2332
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 246ms/step - accuracy: 0.8919 - loss: 0.2989 - val_accuracy: 0.9249 - val_loss: 0.2165
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 246ms/step - accuracy: 0.8610 - loss: 0.3420 - val_accuracy: 0.8935 - val_loss: 0.2721
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 253ms/step - accuracy: 0.8751 - loss: 0.3314 - val_accuracy: 0.9201 - val_loss: 0.2079
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 250ms/step - accuracy: 0.8821 - loss: 0.3261 - val_accuracy: 0.9322 - val_loss: 0.2075
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 249ms/step - accuracy: 0.8802 - loss: 0.3423 - val_accuracy: 0.924

...

Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 16s 249ms/step - accuracy: 0.8034 - loss: 0.7963 - val_accuracy: 0.9007 - val_loss: 0.3049
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 248ms/step - accuracy: 0.8602 - loss: 0.4385 - val_accuracy: 0.8983 - val_loss: 0.3580
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 250ms/step - accuracy: 0.8770 - loss: 0.3775 - val_accuracy: 0.9201 - val_loss: 0.2430
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 249ms/step - accuracy: 0.8661 - loss: 0.3776 - val_accuracy: 0.9249 - val_loss: 0.2070
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 249ms/step - accuracy: 0.8606 - loss: 0.3819 - val_accuracy: 0.9201 - val_loss: 0.2157
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 257ms/step - accuracy: 0.8736 - loss: 0.3466 - val_accuracy: 0.9201 - val_loss: 0.2080
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 248ms/step - accuracy: 0.8960 - loss: 0.2717 - val_accuracy: 0.9249 - val_loss: 0.2209
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 249ms/step - accuracy: 0.8827 - loss: 0.3135 - val_accu

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 16s 251ms/step - accuracy: 0.8043 - loss: 0.7881 - val_accuracy: 0.8959 - val_loss: 0.3598
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 251ms/step - accuracy: 0.8430 - loss: 0.4762 - val_accuracy: 0.8959 - val_loss: 0.6140
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 252ms/step - accuracy: 0.8459 - loss: 0.5671 - val_accuracy: 0.9031 - val_loss: 0.2926
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 256ms/step - accuracy: 0.8719 - loss: 0.3778 - val_accuracy: 0.9153 - val_loss: 0.2434
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 248ms/step - accuracy: 0.8775 - loss: 0.3980 - val_accuracy: 0.9056 - val_loss: 0.2519
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 248ms/step - accuracy: 0.8780 - loss: 0.3542 - val_accuracy: 0.9153 - val_loss: 0.2175
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 250ms/step - accuracy: 0.8770 - loss: 0.4079 - val_accuracy: 0.9080 - val_loss: 0.2887
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 257ms/step - accuracy: 0.8868 - loss: 0.3600 - val_accu

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 16s 249ms/step - accuracy: 0.8261 - loss: 0.7712 - val_accuracy: 0.8983 - val_loss: 0.2762
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 251ms/step - accuracy: 0.8680 - loss: 0.4353 - val_accuracy: 0.9056 - val_loss: 0.3094
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 254ms/step - accuracy: 0.8723 - loss: 0.3645 - val_accuracy: 0.9031 - val_loss: 0.2326
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 250ms/step - accuracy: 0.8795 - loss: 0.3349 - val_accuracy: 0.9419 - val_loss: 0.1969
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 14s 245ms/step - accuracy: 0.8746 - loss: 0.3238 - val_accuracy: 0.9249 - val_loss: 0.1997
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 246ms/step - accuracy: 0.8646 - loss: 0.3509 - val_accuracy: 0.9395 - val_loss: 0.1999
Epoch 7/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 250ms/step - accuracy: 0.8810 - loss: 0.3086 - val_accuracy: 0.9322 - val_loss: 0.1920
Epoch 8/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 245ms/step - accuracy: 0.8934 - loss: 0.2990 - val_accu

In [ ]:
print("\n\n=== Classification Report with Mean±Std ===")

# for class
max_class_length = max(len(str(cls)) for cls in class_names)
print(f"\n{'Class':<{max_class_length}} {'Precision':<6} {'Recall':<6} {'F1-score':<6}  Support")
for idx, cls in enumerate(class_names):
    prec_mean = np.mean(metrics['classes'][cls]['precision'])
    prec_std = np.std(metrics['classes'][cls]['precision'])
    rec_mean = np.mean(metrics['classes'][cls]['recall'])
    rec_std = np.std(metrics['classes'][cls]['recall'])
    f1_mean = np.mean(metrics['classes'][cls]['f1'])
    f1_std = np.std(metrics['classes'][cls]['f1'])
    
    print(f"{idx:<{max_class_length}} "
          f"{prec_mean:.2f}±{prec_std:.2f}  "
          f"{rec_mean:.2f}±{rec_std:.2f}  "
          f"{f1_mean:.2f}±{f1_std:.2f}  "
          f"{test_generator.classes.tolist().count(idx)}")
# for all
def print_avg_row(name, metric_dict):
    prec = f"{np.mean(metric_dict['precision']):.2f}±{np.std(metric_dict['precision']):.2f}"
    rec = f"{np.mean(metric_dict['recall']):.2f}±{np.std(metric_dict['recall']):.2f}"
    f1 = f"{np.mean(metric_dict['f1']):.2f}±{np.std(metric_dict['f1']):.2f}"
    print(f"{name:<{max_class_length}} {prec:<12} {rec:<12} {f1:<12} ")

total_samples = len(test_generator.classes)
print(f"\n{'accuracy':<{max_class_length}} {'':<12} {'':<12} "
      f"{np.mean(metrics['accuracy']):.2f}±{np.std(metrics['accuracy']):.2f}  "
      f"{total_samples}")
print_avg_row('macro avg', metrics['macro_avg'])
print_avg_row('weighted avg', metrics['weighted_avg'])



=== Classification Report with Mean±Std ===

Class Precision Recall F1-score  Support
0 0.44±0.16  0.36±0.17  0.37±0.12  31
1 0.97±0.01  0.98±0.01  0.98±0.00  746
2 0.57±0.05  0.57±0.19  0.56±0.12  65
3 0.42±0.17  0.37±0.16  0.37±0.15  30

accuracy                           0.91±0.01  872
macro avg 0.60±0.08    0.57±0.06    0.57±0.07    
weighted avg 0.90±0.02    0.91±0.01    0.90±0.01    
